<a href="https://colab.research.google.com/github/psygrammer/psypy_lm/blob/main/notebooks/ch02/02_fine_tuning_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Fine-Tuning BERT Models

* 싸이그래머 / 싸이파이 [1]
* 김무성

------------

In [40]:
!git clone https://github.com/psygrammer/psypy_lm.git

Cloning into 'psypy_lm'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 53 (delta 15), reused 15 (delta 2), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [41]:
ls

psypy_lm/  sample_data/


In [42]:
# change work_dir
%cd /content/psypy_lm/notebooks/ch02

/content/psypy_lm/notebooks/ch02


In [43]:
ls

02_fine_tuning_bert.ipynb  in_domain_train.tsv  out_of_domain_dev.tsv


In [44]:
#@title Activating the GPU
# Main menu->Runtime->Change Runtime Type
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [ ]:
 #@title Installing the Hugging Face PyTorch Interface for Bert
!pip install -q transformers

     |████████████████████████████████| 2.0MB 21.1MB/s 
     |████████████████████████████████| 890kB 51.8MB/s 
     |████████████████████████████████| 3.2MB 54.6MB/s 


In [45]:
#@title Importing the modules
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler,SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup

In [46]:
from tqdm import tqdm, trange

In [47]:
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

In [48]:
#@title Specify CUDA as device for Torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [49]:
#@title Loading the Dataset
#source of dataset : https://nyu-mll.github.io/CoLA/
df = pd.read_csv("in_domain_train.tsv", delimiter='\t', header=None,
names=['sentence_source', 'label', 'label_notes', 'sentence'])
df.shape

(8551, 4)

In [50]:
df.sample(10)

,sentence_source,label,label_notes,sentence
3373,l-93,1,NaN,the ball rolled down the hill .
6759,m_02,1,NaN,she said that into the room came aunt norris .
6748,m_02,1,NaN,"if emma had left hartfield , mr woodhouse woul..."
8537,ad03,1,NaN,we believed aphrodite to be omnipotent .
5088,ks08,1,NaN,what is to come is in this document .
2065,rhl07,1,NaN,she fell in love .
7363,sks13,1,NaN,i put the book on the desk on sunday .
3152,l-93,1,NaN,sharon shivered from fear .
1023,bc01,1,NaN,someone from every city hates it .
5803,c_13,0,*,two or boring books take a very long time to r...


In [51]:
#@ Creating sentence, label lists and adding Bert tokens
sentences = df.sentence.values

In [53]:
len(sentences)

8551

In [54]:
sentences

array(["our friends wo n't buy this analysis , let alone the next one we propose .",
       "one more pseudo generalization and i 'm giving up .",
       "one more pseudo generalization or i 'm giving up .", ...,
       'it is easy to slay the gorgon .',
       'i had the strangest feeling that i knew you .',
       'what all did you get for christmas ?'], dtype=object)